In [2]:
%%capture
!pip install swiplserver
!sudo apt-get install swi-prolog -y

In [3]:
w = open("pheptoan.pl", "wt", encoding="utf8")
content = """
:-op(1000,yfx,x).
:-op(950,yfx,@).
:-op(900,yfx,=>).
:-op(850,yfx,v).
:-op(800,yfx,&).
:-op(750,fy,~).

~BT1:-not(BT1).
BT1 & BT2:-BT1, BT2.
BT1 v BT2:-BT1; BT2.
BT1 => BT2:-not(BT1),!; BT2.
tồn_tại(X,BT1):-hằng(X),BT1.
với_mọi(X,BT1):- ~tồn_tại(X,~BT1).
"""
w.write(content)
w.close()

In [4]:
w = open("tuvung.pl", "wt", encoding="utf8")
content = """
từ_vựng(minh, 0).
từ_vựng(lan, 0).
từ_vựng(sinh_viên, 1).
từ_vựng(cao_đẳng, 1).
từ_vựng(đại_học, 1).
từ_vựng(ngành, 1).
từ_vựng(laptop, 1).
từ_vựng(bộ_phận, 1).
từ_vựng(ốm,1).
từ_vựng(thấp, 1).
từ_vựng(cao,1).
từ_vựng(có, 2).
từ_vựng(thích, 2).
từ_vựng(thuộc, 2).
từ_vựng(bạn, 2).
từ_vựng(trường, 1).
"""
w.write(content)
w.close()

In [5]:
w = open("mohinh.pl", "wt", encoding="utf8")
content = """
hằng(c_laptop).
hằng(touchpad).
hằng(uit).
hằng(us).
hằng(cs).
hằng(is).
hằng(X):-từ_vựng(X,0).
sinh_viên(minh).
sinh_viên(lan).
trường(uit).
trường(us).
đại_học(uit).
cao_đẳng(us).
ngành(cs).
ngành(is).
laptop(c_laptop).
bộ_phận(touchpad).
thấp(lan).
cao(minh).
thuộc(minh, us).
thuộc(lan, uit).
có(uit, cs).
có(us, is).
có(lan, c_laptop).
có(c_laptop, touchpad).
bạn(minh, lan).
bạn(lan, minh).

thích(minh, c_laptop).
"""
w.write(content)
w.close()

In [6]:
w = open("parser.pl", "wt", encoding="utf8")
content = """
% --- CẤU TRÚC CÂU (S) ---
s(NP@VP) --> np(NP), vp(VP).
s(NP@VP) --> np(NP), vp(VP), [không], [?].
s(NP@VP) --> np(NP), vp(VP), [?].

% --- DANH TỪ (NP) ---

% 1. Cụm danh từ quan hệ: "Bạn của Lan"
% [CẬP NHẬT] Thêm (X, người) vào vũ trụ DRS để tạo lượng từ 'tồn_tại(X)'
np(lambda(P, Name @ lambda(Y, drs([(X, người)], [RelTerm]) x P@X))) -->
    nn_rel(RelAtom), [của], nnp(Name),
    {RelTerm =.. [RelAtom, X, Y]}.

% 2. Cụm danh từ sở hữu vật chất: "Laptop của Lan"
% Noun@Item giờ đây đã chứa khai báo biến (xem phần nn bên dưới)
% Kết hợp: (Noun@Item) chứa biến + quan hệ 'có' -> tạo ra logic hoàn chỉnh.
np(lambda(P, Name @ lambda(Owner, (Noun@Item) x drs([], [có(Owner, Item)]) x P@Item))) --> nn(Noun), [của], nnp(Name).

% 3. Tân ngữ nghi vấn: "ngành nào"
np(lambda(P, (N@X) x P@X)) --> nn(N), [nào].

% 4. Các cấu trúc khác
np(lambda(P, lambda(X, drs([], [cao_đẳng(X)]) x P@X))) --> [trường, cao, đẳng].
np(lambda(P, lambda(X, drs([], [đại_học(X)]) x P@X))) --> [trường, đại, học].
np(lambda(P, N2 @ lambda(X, (N1@X) x P@X))) --> nn(N1), nnp(N2).
np(lambda(X, (N1@X) x (N2 @ lambda(Y, drs([], [thuộc(X,Y)]))))) --> nn(N1), [thuộc], nnp(N2).
np(lambda(P, lambda(X, (N1@X) x (Name @ lambda(S, drs([], [thích(S,X)]))) x P@X))) --> nn(N1), [mà], nnp(Name), [thích].

% Danh từ đơn
np(N) --> nnp(N).
np(N) --> nn(N).
np(N) --> prp(N).

% --- ĐỘNG TỪ (VP) ---
vp(lambda(X, drs([], [có(_, X)]))) --> [là, của, ai].
vp(lambda(X, drs([], [có(X, _)]))) --> [có, gì].
vp(V@J) --> vb(V), jj(J).
vp(V@N) --> vb(V), np(N).

% --- ĐỘNG TỪ (VB) ---
vb(lambda(P,P)) -->[thì];[là];[có,là];[có,phải,là].

% Xử lý "có thích" và "thích"
vb(lambda(N, lambda(S, N @ lambda(O, drs([], [thích(S, O)]))))) --> [thích].
vb(lambda(N, lambda(S, N @ lambda(O, drs([], [thích(S, O)]))))) --> [có], [thích].

% Động từ CÓ
vb(lambda(N, lambda(S, N @ lambda(O, drs([], [có(S, O)]))))) --> [có].
% Động từ HỌC
vb(lambda(N, lambda(S, N @ lambda(O, drs([], [thuộc(S, O)]))))) --> [học].

% --- TÊN RIÊNG (NNP) ---
% Tên riêng là hằng, không cần thêm vào vũ trụ lượng từ (hoặc đã có cơ chế riêng)
nnp(lambda(P, drs([(lan, người)],[]) x P@lan)) -->[lan].
nnp(lambda(P, drs([(minh, người)],[]) x P@minh)) -->[minh].
nnp(lambda(P, drs([],[]) x P@uit)) -->[uit].
nnp(lambda(P, drs([],[]) x P@cs)) -->[khmt].
nnp(lambda(P, drs([],[]) x P@us)) -->[us].

% --- TÍNH TỪ (JJ) ---
jj(lambda(P, drs([],[cao(P)]))) --> [cao].
jj(lambda(P, drs([],[thấp(P)]))) --> [thấp].

% --- DANH TỪ QUAN HỆ (NN_REL) ---
nn_rel(bạn) --> [bạn].

% --- DANH TỪ CHUNG (NN) ---
% [QUAN TRỌNG] Thêm biến P vào vũ trụ DRS: drs([(P, vật/người)], ...)
% Điều này giúp drs2logic chuyển thành: tồn_tại(P, ...)
nn(lambda(P, drs([(P, người)], [sinh_viên(P)]))) --> [sinh, viên].
nn(lambda(P, drs([(P, vật)], [laptop(P)]))) --> [laptop].
nn(lambda(P, drs([(P, trường)], [đại_học(P)]))) --> [trường, đại, học].
nn(lambda(P, drs([(P, ngành)], [ngành(P)]))) --> [ngành].
nn(lambda(P, drs([(P, trường)], [cao_đẳng(P)]))) --> [trường, cao, đẳng].
nn(lambda(P, drs([(P, trường)], [trường(P)]))) --> [trường].
nn(lambda(P, lambda(X, P@ lambda(Y, drs([],[thích(X,Y)]))))) --> [thích].
nn(lambda(P, lambda(X, P@ lambda(Y, drs([],[có(X,Y)]))))) --> [có].

% --- ĐẠI TỪ (PRP) ---
prp(lambda(P, alpha((X, _), drs([(X, _)],[])) x (P@X))) --> [nó].
prp(lambda(P, lambda(X, drs([(X, người)], []) x P@X))) -->[ai].
"""
w.write(content)
w.close()

In [41]:
w = open("parser.pl", "wt", encoding="utf8")
content = """
% --- CẤU TRÚC CÂU (S) ---
s(NP@VP) --> np(NP), vp(VP).
s(NP@VP) --> np(NP), vp(VP), [không], [?].
s(NP@VP) --> np(NP), vp(VP), [?].
s(NP @ lambda(_, drs([],[]))) --> np(NP), [?].

% --- DANH TỪ (NP) ---

% 1. Cấu trúc "Người vừa VP1 vừa VP2"
% Logic: Noun(X) & VP1(X) & VP2(X) & P(X)
np(lambda(P, (N@X) x (V1@X) x (V2@X) x P@X)) -->
    nn(N), [vừa], vp(V1), [vừa], vp(V2).

% 2. Quan hệ mở rộng: "Bạn của [NP Phức]"
np(lambda(P, NP2 @ lambda(Y, drs([(X, người)], [RelTerm]) x P@X))) -->
    nn_rel(RelAtom), [của], np(NP2),
    {RelTerm =.. [RelAtom, X, Y]}.

% 3. Cụm danh từ ghép: "Sinh viên đại học UIT"
np(lambda(P, N2 @ lambda(Y, (N1@X) x drs([], [thuộc(X,Y)]) x P@X))) -->
    nn(N1), [đại, học], nnp(N2).

% 4. Xử lý "một chiếc laptop"
np(lambda(P, (N@X) x P@X)) --> [một], [chiếc], nn(N).
np(lambda(P, (N@X) x P@X)) --> [một], nn(N).

% 5. Các luật cũ
np(lambda(P, (Target@X) x (Subject @ lambda(S, drs([], [thuộc(S,X)]))) x P@X)) -->
    nn(Target), [nào, mà], np(Subject), [đang, theo, học].

np(lambda(P, (Noun@S) x (ObjNP @ lambda(O, drs([], [có(S,O)]))) x P@S)) -->
    nn(Noun), [có], np(ObjNP).

np(lambda(P, (N1@X) x (Name @ lambda(S, drs([], [thích(S,X)]))) x P@X)) -->
    nn(N1), nnp(Name), [thích].

np(lambda(P, Name @ lambda(Owner, (Noun@Item) x drs([], [có(Owner, Item)]) x P@Item))) -->
    nn(Noun), [của], nnp(Name).

np(lambda(P, (N@X) x P@X)) --> nn(N), [nào].
np(lambda(P, lambda(X, drs([(X, trường)], [cao_đẳng(X)]) x P@X))) --> [trường, cao, đẳng].
np(lambda(P, lambda(X, drs([(X, trường)], [đại_học(X)]) x P@X))) --> [trường, đại, học].

% Danh từ đơn
np(N) --> nnp(N).
np(N) --> nn(N).
np(N) --> prp(N).

% --- ĐỘNG TỪ (VP) ---

% [QUAN TRỌNG - ĐÃ SỬA LỖI] Động từ "là" + NP
% Logic cũ: Tạo ra biến độc lập -> Gây nhập nhằng.
% Logic mới: lambda(S, ... S=X ...). Ép buộc chủ ngữ S phải trùng với biến X của NP.
vp(lambda(S, N @ lambda(X, drs([], [S=X])))) --> [là], np(N).

% Các luật VP khác
vp(lambda(X, drs([], [có(_, X)]))) --> [là, của, ai].
vp(lambda(Subject, drs([(Thing, vật)], [có(Subject, Thing)]))) --> [có, gì].
vp(V@J) --> vb(V), jj(J).
vp(V@N) --> vb(V), np(N).

% --- ĐỘNG TỪ (VB) ---
vb(lambda(P,P)) --> [thì].
vb(lambda(P,P)) --> [có, là].
vb(lambda(P,P)) --> [có, phải, là].

% Động từ "sở hữu"
vb(lambda(N, lambda(S, N @ lambda(O, drs([], [có(S, O)]))))) --> [sở, hữu].

vb(lambda(N, lambda(S, N @ lambda(O, drs([], [thích(S, O)]))))) --> [thích].
vb(lambda(N, lambda(S, N @ lambda(O, drs([], [thích(S, O)]))))) --> [có], [thích].
vb(lambda(N, lambda(S, N @ lambda(O, drs([], [có(S, O)]))))) --> [có].
vb(lambda(N, lambda(S, N @ lambda(O, drs([], [thuộc(S, O)]))))) --> [học].

% --- TÊN RIÊNG (NNP) ---
nnp(lambda(P, drs([],[]) x P@lan)) -->[lan].
nnp(lambda(P, drs([],[]) x P@minh)) -->[minh].
nnp(lambda(P, drs([],[]) x P@uit)) -->[uit].
nnp(lambda(P, drs([],[]) x P@cs)) -->[khmt].
nnp(lambda(P, drs([],[]) x P@us)) -->[us].

% --- TÍNH TỪ (JJ) ---
jj(lambda(P, drs([],[cao(P)]))) --> [cao].
jj(lambda(P, drs([],[thấp(P)]))) --> [thấp].

% --- DANH TỪ QUAN HỆ ---
nn_rel(bạn) --> [bạn].

% --- DANH TỪ CHUNG (NN) ---
nn(lambda(P, drs([(P, người)], []))) --> [người].
nn(lambda(P, drs([(P, người)], [sinh_viên(P)]))) --> [sinh, viên].
nn(lambda(P, drs([(P, vật)], [laptop(P)]))) --> [laptop].
nn(lambda(P, drs([(P, trường)], [đại_học(P)]))) --> [trường, đại, học].
nn(lambda(P, drs([(P, ngành)], [ngành(P)]))) --> [ngành].
nn(lambda(P, drs([(P, trường)], [cao_đẳng(P)]))) --> [trường, cao, đẳng].
nn(lambda(P, drs([(P, trường)], [trường(P)]))) --> [trường].
nn(lambda(P, lambda(X, P@ lambda(Y, drs([],[thích(X,Y)]))))) --> [thích].
nn(lambda(P, lambda(X, P@ lambda(Y, drs([],[có(X,Y)]))))) --> [có].

% --- ĐẠI TỪ (PRP) ---
prp(lambda(P, alpha((X, _), drs([(X, _)],[])) x (P@X))) --> [nó].
prp(lambda(P, lambda(X, drs([(X, người)], []) x P@X))) -->[ai].
"""
w.write(content)
w.close()

In [7]:
w = open("lambda.pl", "wt", encoding="utf8")
content = """
biến_đổi_b(A, KQ):-
	var(A),!,KQ=A.
biến_đổi_b(F@A,KQ):-
	compound(F),biến_đổi_b(F,KQ1),
	triển_khai(KQ1,A,KQ2),!,
	biến_đổi_b(KQ2,KQ).
biến_đổi_b(F,KQ):-
	F=..[A|B],biến_đổi_ds(B,KQ1),
	KQ=..[A|KQ1].

biến_đổi_ds([],[]).
biến_đổi_ds([F|A],[KQ|B]):-
	biến_đổi_b(F,KQ),
	biến_đổi_ds(A,B).

triển_khai(lambda(X,F),A,KQ):-
	thay_thế(A,X,F,KQ).

thay_thế(A,B,F,KQ):-F==B,!,KQ=A.
thay_thế(_,_,F,KQ):-
	not(compound(F)),!,KQ=F.
thay_thế(A,B,F,KQ):-
	F=..[H|[B1|F1]],
	member(H,[lamda,với_mọi,tất_cả]),!,
	(
		B1==B,!,KQ=F;

		thay_thế(A,B,F1,KQ1),
		KQ=..[H|[B1|KQ1]]
	).
thay_thế(A,B,F,KQ):-
	F=..[H|B1],thay_thế_ds(A,B,B1,KQ1),
	!,KQ=..[H|KQ1].

thay_thế_ds(_,_,[],[]).
thay_thế_ds(A,B,[F|T1],[KQ|T2]):-
	thay_thế(A,B,F,KQ),
	thay_thế_ds(A,B,T1,T2).

dịch(S,KQ):-
	s(T,S,[]),biến_đổi_b(T,KQ).
"""
w.write(content)
w.close()

In [8]:
w = open("kethop.pl", "wt", encoding="utf8")
content = """
sởchỉ([X|_],Y,X):- unifiable(X, Y, _).
sởchỉ([_|T],Y,X):- sởchỉ(T, Y, X).

danhsáchdrs(drs(A,B), [drs(A,B)]).
danhsáchdrs(alpha(X,A), [alpha(X,A)]).
danhsáchdrs(lambda(X,A), [lambda(X,A)]).
danhsáchdrs(A x B, R):-danhsáchdrs(A, RA), danhsáchdrs(B,RB),!, append(RA, RB, R).


kếthợp(drs(D,C1),drs(D,C2)):-
	kếthợp(C1,C2).

kếthợp(alpha(X, drs(D,C1)), alpha(X, drs(D,C2))):-
	kếthợp(C1,C2).

kếthợp(B1 x alpha((X,XB), B2), drs(D3,C3)):-
	kếthợp(B1,drs(D1,C1)),
kếthợp(B2,drs(D2,C2)),
	append(D2,D1,D3t),sởchỉ(D1, (X,XB), (V,_)),append(C1, [X=V], C1t),append(C1t,C2,C3t), list_to_set(D3t, D3), list_to_set(C3t, C3).

kếthợp(B1 x B2,drs(D3,C3)):-
	kếthợp(B1,drs(D1,C1)),
kếthợp(B2,drs(D2,C2)),
	append(D2,D1,D3t),append(C1,C2,C3t), list_to_set(D3t, D3), list_to_set(C3t, C3).

kếthợp([B1 => B2|C1],[B3 => B4|C2]):-
	!, kếthợp(B1,B3), kếthợp(B2,B4),
	kếthợp(C1,C2).
kếthợp([B1 v B2|C1],[B3 v B4|C2]):-
	!,kếthợp(B1,B3),kếthợp(B2,B4),
	kếthợp(C1,C2).

kếthợp([~ B1|C1], [~ B2|C2]):-
	!,kếthợp(B1,B2),kếthợp(C1,C2).

kếthợp([C|C1],[C|C2]):-kếthợp(C1,C2).

kếthợp([],[]).

kếthợpcâu(drs(P,Q), [], drs(P,Q)).
kếthợpcâu(drs(P,Q), [B1|T] ,drs(D3,C3)):-
	kếthợp(drs(P,Q) x B1, R1), kếthợpcâu(R1, T, drs(D3, C3)).

kếthợpcâu(lambda(X, drs(P,Q)), [B1|T], lambda(X, drs(D3,C3))):-
	kếthợp(drs(P,Q) x B1, R1), kếthợpcâu(R1, T, drs(D3, C3)).

kếthợpcâu(drs(P,Q), [lambda(X, B1)|T], lambda(X, drs(D3,C3))):-
	kếthợp(drs(P,Q) x B1, R1), kếthợpcâu(R1, T, drs(D3, C3)).

dịchvănbản(A,[],A):-!.
dịchvănbản(P,[S|T],R):- s(Rs, S, []), biến_đổi_b(Rs, E), danhsáchdrs(E, Re), kếthợpcâu(P, Re, Rp), dịchvănbản(Rp, T, R), !.

"""
w.write(content)
w.close()

In [44]:
w = open("drs2logic.pl", "wt", encoding="utf8")
content = """
drs2logic(A,A):-A=..[R|_], từ_vựng(R,_).
drs2logic(A,A):-A=..[=|_].

% Quy tắc cơ bản
drs2logic(drs([],[drs([(X,_)],A) => B|T]), với_mọi(X, (Ra => Rb) & Rt)):-drs2logic(A,Ra), drs2logic(B, Rb), drs2logic(T, Rt).
drs2logic(drs([],[drs([(X,_)|TX],A) => B|T]), với_mọi(X, R)):-drs2logic(drs([],[drs(TX,A) => B|T]), R).

% [MỚI] Tự động rút gọn khi gặp X=Y trong DRS
% Nếu trong danh sách điều kiện có X=Y, thực hiện đồng nhất (Unify) X với Y ngay lập tức
drs2logic(drs([], [X=Y | T]), R) :-
    X = Y,  % Lệnh này hợp nhất 2 biến trong Prolog
    drs2logic(drs([], T), R). % Tiếp tục xử lý phần còn lại

drs2logic(drs([],[A]), R):-drs2logic(A,R).
drs2logic(drs([],[A|T]), Ra & Rt):- drs2logic(A,Ra), drs2logic(drs([],T),Rt).

% Xử lý lượng từ tồn tại
drs2logic(drs([(A,_)],B), tồn_tại(A, R)):-var(A),!,drs2logic(drs([],B), R).
drs2logic(drs([(_,_)],B), R):-drs2logic(drs([],B), R).
drs2logic(drs([(A,_)|T], B), tồn_tại(A, R)):-var(A),!,drs2logic(drs(T,B), R).
drs2logic(drs([(_,_)|T], B), R):-drs2logic(drs(T,B), R).

drs2logic(lambda(X, A), lambda(X, R)):- drs2logic(A,R).
"""
w.write(content)
w.close()

In [45]:
from swiplserver import PrologMQI

mqi = PrologMQI()
prolog = mqi.create_thread()
result = prolog.query("set_prolog_flag(encoding,utf8).")

result = prolog.query("consult(\"pheptoan.pl\").")
result = prolog.query("consult(\"tuvung.pl\").")
result = prolog.query("consult(\"mohinh.pl\").")
result = prolog.query("consult(\"parser.pl\").")
result = prolog.query("consult(\"lambda.pl\").")
result = prolog.query("consult(\"kethop.pl\").")
result = prolog.query("consult(\"drs2logic.pl\").")


In [11]:
"""
UIT có phải trường đại học không ?
Lan có là sinh viên UIT không ?
UIT có ngành KHMT không ?
"""
result = prolog.query("đại_học(us).")
print(result)
result = prolog.query("thuộc(lan, uit).")
print(result)
result = prolog.query("có(uit, cs).")
print(result)


False
True
True


In [12]:
def toString(d):
  OPS = ["&", "v", "=>", "<=>", "@" , "x", ",", "="]
  if type(d) is dict: # đầu vào là một biểu thức
    functor = d.get("functor")
    args = d.get("args")
    bInf = (functor == "=>" or functor == "<=>" or functor == "==")
    if functor in OPS: # biểu thức logic hoặc biểu thức lambda
      st = []
      if bInf: # biểu thức logic với phép kéo theo
        st = ["({})".format(toString(args[0])), "{}".format(functor)]
        for e in args[1:]:
          st.append("({})".format(toString(e)))
      else: # biểu thức với phép hội, tuyển, và triển khai.
        if functor == ",":
          st = [toString(args[0])]
          for e in args[1:]:
            st.append(toString(e))
          st = tuple(st)
          return "{}".format(st).replace("'", "")
        else:
          st = [toString(args[0]), "{}".format(functor)]
          for e in args[1:]:
            st.append(toString(e))
          return " ".join(st)
    else: # biểu thức khác
      st = ["{}({}".format(functor, toString(args[0]))]
      for e in args[1:]:
        st.append("{}".format(toString(e)))
      return ",".join(st) + ")"
  else: # đầu vào là một giá trị
    if type(d) is list:
      st = []
      for e in d:
        st.append("{}".format(toString(e)))
      return "{}".format(st).replace("'", "")
      #return "{}".format(d)
    else:
      return "{}".format(d)



In [ ]:
result = prolog.query("dịchvănbản(drs([],[]),[[uit, có, ngành, khmt, không, ?]], DRS), drs2logic(DRS, R).")
qry = toString(result[0]["R"])
print(qry)
print(result[0]["R"])
result = prolog.query(qry)
print(result)

ngành(cs) & có(uit,cs)
{'args': [{'args': ['cs'], 'functor': 'ngành'}, {'args': ['uit', 'cs'], 'functor': 'có'}], 'functor': '&'}
True


In [ ]:
result = prolog.query("dịchvănbản(drs([],[]),[[laptop, mà, minh, thích, có, gì, ?]], lambda(X, DRS)), drs2logic(DRS, R).")
qry = toString(result[0]["R"])
print(qry)
print(result[0]["R"])
result = prolog.query(qry)
print(result[0]['B'])

tồn_tại(B,tồn_tại(A,laptop(A) & thích(minh,A) & có(A,B)))
{'args': ['B', {'args': ['A', {'args': [{'args': ['A'], 'functor': 'laptop'}, {'args': [{'args': ['minh', 'A'], 'functor': 'thích'}, {'args': ['A', 'B'], 'functor': 'có'}], 'functor': '&'}], 'functor': '&'}], 'functor': 'tồn_tại'}], 'functor': 'tồn_tại'}
touchpad


In [22]:
result = prolog.query("dịchvănbản(drs([],[]),[[bạn, của, lan, học, trường, nào, ?]], DRS), drs2logic(DRS, R).")
qry = toString(result[0]["R"])
print(qry)
print(result[0]["R"])
result = prolog.query(qry)
print(result[0]['A'])

tồn_tại(A,tồn_tại(B,bạn(B,lan) & trường(A) & thuộc(B,A)))
{'args': ['A', {'args': ['B', {'args': [{'args': ['B', 'lan'], 'functor': 'bạn'}, {'args': [{'args': ['A'], 'functor': 'trường'}, {'args': ['B', 'A'], 'functor': 'thuộc'}], 'functor': '&'}], 'functor': '&'}], 'functor': 'tồn_tại'}], 'functor': 'tồn_tại'}
us


In [23]:
result = prolog.query("dịchvănbản(drs([],[]),[[us, có, ngành, nào, ?]], DRS), drs2logic(DRS, R).")
qry = toString(result[0]["R"])
print(qry)
print(result[0]["R"])
result = prolog.query(qry)
print(result[0]['A'])

tồn_tại(A,ngành(A) & có(us,A))
{'args': ['A', {'args': [{'args': ['A'], 'functor': 'ngành'}, {'args': ['us', 'A'], 'functor': 'có'}], 'functor': '&'}], 'functor': 'tồn_tại'}
is


In [47]:
result = prolog.query("dịchvănbản(drs([],[]),[[bạn, của, lan, có, thích, laptop, của, lan, không, ?]], DRS), drs2logic(DRS, R).")
qry = toString(result[0]["R"])
print(qry)
print(result[0]["R"])
result = prolog.query(qry)
print(result)

tồn_tại(A,tồn_tại(B,bạn(B,lan) & laptop(A) & có(lan,A) & thích(B,A)))
{'args': ['A', {'args': ['B', {'args': [{'args': ['B', 'lan'], 'functor': 'bạn'}, {'args': [{'args': ['A'], 'functor': 'laptop'}, {'args': [{'args': ['lan', 'A'], 'functor': 'có'}, {'args': ['B', 'A'], 'functor': 'thích'}], 'functor': '&'}], 'functor': '&'}], 'functor': '&'}], 'functor': 'tồn_tại'}], 'functor': 'tồn_tại'}
[{'A': 'c_laptop', 'B': 'minh'}]


In [48]:
result = prolog.query("dịchvănbản(drs([],[]),[[trường, đại, học, nào, mà, người, có, laptop, minh, thích, đang, theo, học, ?]], DRS), drs2logic(DRS, R).")
qry = toString(result[0]["R"])
print(qry)
print(result[0]["R"])
result = prolog.query(qry)
print(result)

tồn_tại(A,tồn_tại(B,tồn_tại(C,đại_học(C) & laptop(A) & thích(minh,A) & có(B,A) & thuộc(B,C))))
{'args': ['A', {'args': ['B', {'args': ['C', {'args': [{'args': ['C'], 'functor': 'đại_học'}, {'args': [{'args': ['A'], 'functor': 'laptop'}, {'args': [{'args': ['minh', 'A'], 'functor': 'thích'}, {'args': [{'args': ['B', 'A'], 'functor': 'có'}, {'args': ['B', 'C'], 'functor': 'thuộc'}], 'functor': '&'}], 'functor': '&'}], 'functor': '&'}], 'functor': '&'}], 'functor': 'tồn_tại'}], 'functor': 'tồn_tại'}], 'functor': 'tồn_tại'}
[{'A': 'c_laptop', 'B': 'lan', 'C': 'uit'}]


In [46]:
result = prolog.query("dịchvănbản(drs([],[]),[[ai, là, bạn, của, người, vừa, là, sinh, viên, đại, học, uit, vừa, sở, hữu, một, chiếc, laptop, ?]], lambda(X,DRS)), drs2logic(DRS, R).")
qry = toString(result[0]["R"])
print(qry)
print(result[0]["R"])
result = prolog.query(qry)
print(result)

tồn_tại(B,tồn_tại(C,tồn_tại(D,tồn_tại(D,tồn_tại(A,sinh_viên(D) & thuộc(D,uit) & laptop(C) & có(D,C) & bạn(B,D) & A = B)))))
{'args': ['B', {'args': ['C', {'args': ['D', {'args': ['D', {'args': ['A', {'args': [{'args': ['D'], 'functor': 'sinh_viên'}, {'args': [{'args': ['D', 'uit'], 'functor': 'thuộc'}, {'args': [{'args': ['C'], 'functor': 'laptop'}, {'args': [{'args': ['D', 'C'], 'functor': 'có'}, {'args': [{'args': ['B', 'D'], 'functor': 'bạn'}, {'args': ['A', 'B'], 'functor': '='}], 'functor': '&'}], 'functor': '&'}], 'functor': '&'}], 'functor': '&'}], 'functor': '&'}], 'functor': 'tồn_tại'}], 'functor': 'tồn_tại'}], 'functor': 'tồn_tại'}], 'functor': 'tồn_tại'}], 'functor': 'tồn_tại'}
[{'B': 'minh', 'C': 'c_laptop', 'D': 'lan', 'A': 'minh'}]
